<a href="https://colab.research.google.com/github/profsuccodifrutta/ai_act_RAG_navigator/blob/main/ai_act_RAG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CREATION OF ENVIORMENT

In [3]:
# install uv
!curl -LsSf https://astral.sh/uv/install.sh | sh
import sys
import os
from google.colab import userdata
sys.path.append("/root/.cargo/bin")

# configuratrion github with secrets
GIT_USER = "profsuccodifrutta"
GIT_REPO = "ai_act_RAG_navigator"

try:
    GIT_TOKEN = userdata.get('github_token')
except Exception:
    print("ERROR")
    sys.exit(1)

# clone repo
if not os.path.exists(GIT_REPO):
    !git clone https://{GIT_TOKEN}@github.com/{GIT_USER}/{GIT_REPO}.git
else:
    print(f"folder {GIT_REPO} already exists.")

%cd {GIT_REPO}

!git remote set-url origin https://{GIT_TOKEN}@github.com/{GIT_USER}/{GIT_REPO}.git

if not os.path.exists("pyproject.toml"):
    !uv init
else:
    print("uv already initialized.")

# add libraries
!uv add langchain langchain-community langchain-google-genai faiss-cpu pymupdf spacy python-dotenv
!uv add langchain-text-splitters langchain-core

# install on colab
!uv pip install --system langchain langchain-community langchain-google-genai faiss-cpu pymupdf spacy python-dotenv
!uv pip install --system langchain-text-splitters langchain-core

print("\nEnviorment configuration completed")

downloading uv 0.10.4 x86_64-unknown-linux-gnu
no checksums to verify
installing to /usr/local/bin
  uv
  uvx
everything's installed!
Cloning into 'ai_act_RAG_navigator'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 113 (delta 66), reused 12 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (113/113), 2.38 MiB | 4.23 MiB/s, done.
Resolving deltas: 100% (66/66), done.
/content/ai_act_RAG_navigator
uv already initialized.
Using CPython 3.12.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Resolved 99 packages in 3ms
Prepared 97 packages in 11.71s
Installed 97 packages in 389ms
 + aiohappyeyeballs==2.6.1
 + aiohttp==3.13.3
 + aiosignal==1.4.0
 + annotated-doc==0.0.4
 + annotated-types==0.7.0
 + anyio==4.12.1
 + attrs==25.4.0
 + blis==1.3.3
 + catalogue==2.0.10
 + certifi==2026.1.4
 + cffi==2.0.0
 + charset-normalizer==3.4.4
 + click==8.3.1
 + clou

In [ ]:
# configuration
!git config --global user.email "matteo@drezzo.ch"
!git config --global user.name "profsuccodifrutta"
!git branch -M main

# LOADING LIBRARIES

In [ ]:
!pip install langchain langchain-community langchain-classic rank_bm25 faiss-cpu

In [4]:
from google.colab import drive
import os
import requests
import shutil
import fitz
import re
import pandas as pd
import matplotlib.pyplot as plt
import gc
import random
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
from spacy import displacy
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import plotly.graph_objects as go
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import plotly.io as pio
import plotly.express as px
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score
from scipy.special import softmax
from langchain_classic.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from collections import defaultdict
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense, TextVectorization, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.layers import Bidirectional, Dropout
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

# SETTING UP DRIVE

Creates a local data/ directory within the repository, and loads the official 144-page EU AI Act PDF. It also prepares a dedicated folder on Drive to store the future FAISS vector index

In [2]:
drive.mount('/content/drive')

# local env set up
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Folder '{data_dir}' created.")

# path configuration
drive_pdf_path = "/content/drive/MyDrive/compliance_and_regulation_rag/regulation_pdfs/ai_act_2024.pdf"

local_pdf_path = os.path.join(data_dir, "eu_ai_act.pdf") # local destination

# Copy file from Drive to Local Data folder
if os.path.exists(drive_pdf_path):
    shutil.copy(drive_pdf_path, local_pdf_path)
    print(f"File copied from Drive to: {local_pdf_path} ({os.path.getsize(local_pdf_path)} bytes)")
else:
    print(f"Error: File NOT found on Drive at {drive_pdf_path}")

# persistence folder for FAISS index
drive_persistence_path = "/content/drive/MyDrive/compliance_and_regulation_rag/ai_act_rag_index"
if not os.path.exists(drive_persistence_path):
    os.makedirs(drive_persistence_path)
    print("Folder for index persistence ready on Drive.")

Mounted at /content/drive
Folder 'data' created.
File copied from Drive to: data/eu_ai_act.pdf (2583319 bytes)


# PRE-PROCESSING and EDA

The parse_ai_act_robust() function acts as a highly specialized document parser designed to extract the EU AI Act into clean, semantically isolated legal units. First, it reads the PDF page by page, stripping away formatting noise like document links, page numbers, and line-break hyphens. It employs a multiline regex strategy to detect and truncate footnote blocks at the bottom of the pages; this prevents those footnotes from being falsely extracted as standalone Recitals or corrupting the flow of the main text. After sanitizing the pages, it groups them into "RECITAL", "ARTICLE", and "ANNEX" buckets based on their structural locations in the document. Finally, the function puts the cleaned text back together and splits it only where real section titles appear (like “Article 5” on its own line). It does not split when those words appear inside a sentence (for example, “in accordance with Article 99”). This ensures that each row in the final dataframe contains one complete legal section, making it ideal for use in a RAG system.

In [ ]:
def parse_ai_act_robust(pdf_path):
    doc = fitz.open(pdf_path)

    categories_text = {
        "RECITAL": [],
        "ARTICLE": [],
        "ANNEX": []
    }

    noise_pattern = re.compile(r"(\d+/144|ELI: http://data\.europa\.eu/eli/reg/.+|OJ L, 12\.7\.2024|EN|\(Text with EEA relevance\))")
    footnote_cleaner = re.compile(r"\n\s*\(\s*\d+\s*\)\s*(?:OJ\b|Position\b|Directive\b|Regulation\b|See\b|Case\b).*$", re.IGNORECASE | re.DOTALL)

    footnotes_removed = 0

    for page_num in range(len(doc)):
        text = doc[page_num].get_text("text").replace("-\n", "")

        clean_lines = []
        for line in text.split("\n"):
            cleaned_line = noise_pattern.sub("", line).strip()
            if cleaned_line and not cleaned_line.isdigit():
                clean_lines.append(cleaned_line)

        # Join the page back together
        final_page_text = "\n" + "\n".join(clean_lines)

        if footnote_cleaner.search(final_page_text):
            # Replace the footnote block with a single newline
            final_page_text = footnote_cleaner.sub("\n", final_page_text)
            footnotes_removed += 1

        p_idx = page_num + 1
        if p_idx < 44:
            categories_text["RECITAL"].append(final_page_text)
        elif 44 <= p_idx <= 123:
            categories_text["ARTICLE"].append(final_page_text)
        else:
            categories_text["ANNEX"].append(final_page_text)

    doc.close()
    print(f"Removed footnote blocks from {footnotes_removed} pages.")

    units = []

    # proces recitals
    recital_text = "\n" + "\n".join(categories_text["RECITAL"]) + "\n"
    rec_parts = re.split(r"\n\s*\(\s*(\d+)\s*\)\s+", recital_text)
    for i in range(1, len(rec_parts), 2):
        text_content = rec_parts[i+1].replace("\n", " ").strip()
        if text_content:
            units.append({"id": f"RECITAL {rec_parts[i]}", "category": "RECITAL", "text": text_content})

    # proces articles
    article_text = "\n" + "\n".join(categories_text["ARTICLE"]) + "\n"
    art_parts = re.split(r"(?i)\n\s*Article\s+(\d+)\s*\n", article_text)
    for i in range(1, len(art_parts), 2):
        text_content = art_parts[i+1].replace("\n", " ").strip()
        if text_content:
            units.append({"id": f"ARTICLE {art_parts[i]}", "category": "ARTICLE", "text": text_content})

    # proces annexes
    annex_text = "\n" + "\n".join(categories_text["ANNEX"]) + "\n"
    annex_parts = re.split(r"(?i)\n\s*ANNEX\s+([IVXLCDM]+)\s*\n", annex_text)
    for i in range(1, len(annex_parts), 2):
        text_content = annex_parts[i+1].replace("\n", " ").strip()
        if text_content:
            units.append({"id": f"ANNEX {annex_parts[i].upper()}", "category": "ANNEX", "text": text_content})

    return pd.DataFrame(units)

In [ ]:
df_units = parse_ai_act_robust(local_pdf_path)
df_units['len'] = df_units['text'].str.len()

print(f"Clean Units created: {len(df_units)}")
print(f"Max length: {df_units['len'].max()}")

Removed footnote blocks from 23 pages.
Clean Units created: 307
Max length: 17095


An embedding vector represents the "average" meaning of a text. If a chunk is too long and covers five different legal requirements, the vector becomes vague, making it harder for the system to find that specific information during a search. Even if an LLM can read a huge chunk, providing too much irrelevant text alongside the answer creates "noise." This increases the risk of the model hallucinating or missing the specific detail you're looking for. Later on the lenght of the chunks will be calibrated, below is shown a visual inspection of the actual distribution.

In [ ]:
avg_len = df_units['len'].mean()

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=df_units['len'],
    nbinsx=50,
    marker_color='#e74c3c',
    marker_line=dict(color='white', width=1),
    name='Unit Lengths'
))

fig.add_vline(x=1500, line_dash="dash", line_color="#c0392b")

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    line=dict(color='#c0392b', dash='dash'),
    name='Target RAG Limit (1500)'
))

fig.add_vline(x=avg_len, line_dash="solid", line_color="#f39c12")

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    line=dict(color='#f39c12', shape='linear'),
    name=f'Average Length ({avg_len:.0f})'
))

fig.update_layout(
    title='<b>Distribution of Legal Unit Lengths </b>',
    xaxis_title="Characters",
    yaxis_title="Number of Units",
    template='plotly_white',
    bargap=0.05,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="right",
        x=0.99,
        bgcolor="rgba(255, 255, 255, 0.5)"
    ),
    height=500
)


fig.show()

# METADATA ENRICHMENT WITH ERROR ANALYSIS

**NER FOR ENRICHING THE METADATA**


In a standard RAG system, the database relies entirely on dense vector embeddings to find relevant information. While vectors are excellent at understanding the semantic meaning or general theme of a question, they struggle significantly with exact keyword matching. For example, a vector search might easily confuse the legal obligations of a "Provider" with those of a "Deployer" because both concepts occupy a very similar mathematical space in the vector database.

**ERROR ANALYSIS METHODOLOGY**


Identifying and extracting these entities requires an iterative methodology that we'll call Error Analysis, as relying entirely on pre-trained machine learning models, this case of domain-specific legal text will lead to systemic failures. Pre-trained NLP models calculate statistical probabilities based on everyday internet text, they get easily confused by the dense capitalization, roman numerals, and acronyms found in the AI Act.

To overcome this, we established an iterative feedback loop:


1.   Baseline Definition: We injected a baseline set of deterministic rules (the EntityRuler) into the pipeline before the machine learning component.
2.   Visual Diagnostics: We ran random samples of the text through the pipeline and used a visualizer (displaCy) to inspect the model's behavior.
3. Identifying False Positives and Negatives: By reading the annotated text, we spotted areas where the model missed critical terms.
4. Rule Calibration: We updated our deterministic rules to either explicitly classify these edge cases (e.g. assigning "Annexes" to DOCUMENT)


The rules that drive this extraction are defined in a simple but highly powerful list called patterns. Every rule is a dictionary containing two key-value pairs: the "label" (which dictates the category, like "ORG" or "RISK_LEVEL") and the "pattern" (which defines what text to look for).

Afther the pattern have been identified they are added to the document object of langchain library, since could be useful to exploit these information while retriving, possibly increasing the rag system performance.

In [ ]:
nlp = spacy.load("en_core_web_sm")

ruler = nlp.add_pipe("entity_ruler", before="ner") # ner = spacy module for ner
                                                   # before the model prediction

patterns = [
    # organizations and authoroties
    {"label": "ORG", "pattern": "AI Office"},
    {"label": "ORG", "pattern": "European Artificial Intelligence Board"},
    {"label": "ORG", "pattern": "Market Surveillance Authority"},
    {"label": "ORG", "pattern": [{"lower": "european"}, {"lower": "commission"}]},
    {"label": "ORG", "pattern": "European Data Protection Board"},
    {"label": "ORG", "pattern": "European Data Protection Supervisor"},
    {"label": "ORG", "pattern": "EDPB"},
    {"label": "ORG", "pattern": "EDPS"},
    {"label": "ORG", "pattern": "Board"},
    {"label": "ORG", "pattern": "Commission"},

    # geopolitical entities
    {"label": "GPE", "pattern": "Union"},
    {"label": "GPE", "pattern": "EU"},

    # documents
    {"label": "DOCUMENT", "pattern": "Regulation"},
    {"label": "DOCUMENT", "pattern": "Directive"},
    {"label": "DOCUMENT", "pattern": [{"lower": "annex"}, {"IS_ALPHA": True, "IS_UPPER": True}]},
    {"label": "DOCUMENT", "pattern": [{"lower": "annexes"}, {"IS_ALPHA": True, "IS_UPPER": True}]},

    # Chapters
    {"label": "LAW", "pattern": [{"lower": "chapter"}, {"IS_DIGIT": True}]},
    {"label": "LAW", "pattern": [{"lower": "chapter"}, {"IS_ALPHA": True, "IS_UPPER": True}]},

    # Sections
    {"label": "LAW", "pattern": [{"lower": "section"}, {"IS_DIGIT": True}]},
    {"label": "LAW", "pattern": [{"lower": "section"}, {"IS_ALPHA": True, "IS_UPPER": True}]},
    {"label": "LAW", "pattern": [{"lower": "sections"}, {"IS_DIGIT": True}]},
    {"label": "LAW", "pattern": [{"lower": "sections"}, {"IS_ALPHA": True, "IS_UPPER": True}]},

    # Articles
    {"label": "LAW", "pattern": [{"lower": "article"}, {"IS_DIGIT": True}]},

    # ignore these patterns
    {"label": "IGNORE", "pattern": "AI"},
    {"label": "IGNORE", "pattern": [{"lower": "artificial"}, {"lower": "intelligence"}]},
    {"label": "IGNORE", "pattern": [{"lower": "ai"}, {"lower": "system"}]},

    # concepts and legal roles
    {"label": "CONCEPT", "pattern": "CE"},
    {"label": "CONCEPT", "pattern": [{"text": "CE"}, {"lower": "marking"}]},
    {"label": "CONCEPT", "pattern": [{"lower": "general"}, {"IS_PUNCT": True, "OP": "?"}, {"lower": "purpose"}, {"lower": "ai"}]},
    {"label": "LEGAL_ROLE", "pattern": [{"lower": "provider"}]},
    {"label": "LEGAL_ROLE", "pattern": [{"lower": "providers"}]},
    {"label": "LEGAL_ROLE", "pattern": [{"lower": "deployer"}]},
    {"label": "LEGAL_ROLE", "pattern": [{"lower": "deployers"}]},
    # risk levels
    {"label": "RISK_LEVEL", "pattern": [{"lower": "high"}, {"IS_PUNCT": True, "OP": "?"}, {"lower": "risk"}]},
    {"label": "RISK_LEVEL", "pattern": [{"lower": "unacceptable"}, {"lower": "risk"}]},
    {"label": "RISK_LEVEL", "pattern": [{"lower": "limited"}, {"lower": "risk"}]},
    {"label": "RISK_LEVEL", "pattern": [{"lower": "minimal"}, {"lower": "risk"}]},
]

ruler.add_patterns(patterns)

**LANGCHAIN DOCOMUNENT OBJECT EXPANSION**


LangChain wraps every chunk of the text into a specific Python object called a Document, it has exactly two parts:


1.   page_content: A single string of text
2.   metadata: A standard Python Dictionary

RecursiveCharacterTextSplitter() slices the text into chunks of up to 1500 characters, maintaining a 200-character overlap to ensure no nuanced legal context is lost at the boundaries. It applies "contextual chunking" by prepending the exact legal identifier (such as [ARTICLE 10]) directly into the text of every single slice, this guarantees that the LLM always knows exactly which part of the law it is referencing. Then it packages each segment into a LangChain Document object and enriches it with structured metadata, including the category, legal unit, and a flag indicating if it is a continuation resulting in a robust, searchable dataset ready for embedding into the FAISS vector database.

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", ".", " ", ""]
)

final_documents = []


for _, row in df_units.iterrows():
    unit_id = row['id']
    category = row['category']
    text = row['text']

    chunks = text_splitter.split_text(text)

    for i, chunk in enumerate(chunks):
        enriched_content = f"[{unit_id}] {chunk}"

        combined_metadata = {
            "source": "EU_AI_Act_2024",
            "category": category,
            "legal_unit": unit_id,
            "chunk_index": i,
            "is_continuation": i > 0
        }

        spacy_doc = nlp(enriched_content)
        extracted = defaultdict(set)

        for ent in spacy_doc.ents:
            if ent.label_ != "IGNORE":
                extracted[ent.label_].add(ent.text)

        for label, entity_set in extracted.items():
            combined_metadata[f"ner_{label}"] = list(entity_set)

        doc = Document(
            page_content=enriched_content,
            metadata=combined_metadata
        )

        final_documents.append(doc)

print(f"Total vectorized chunks: {len(final_documents)}")

Total vectorized chunks ready for FAISS: 577


Check of the information a single chunk is holding:

In [ ]:
sample_doc = random.choice(final_documents)

for key, value in sample_doc.metadata.items():
    if not key.startswith("ner_"):
        clean_key = key.replace('_', ' ').title()
        print(f"{clean_key}: {value}")


found_entities = False
for key, value in sample_doc.metadata.items():
    if key.startswith("ner_"):
        found_entities = True
        print(f"{key}: {value}")

if not found_entities:
    print("(No entities found in this specific chunk)")

print("\n--- Text Preview ---")
print(f"{sample_doc.page_content[:800]}...")

Source: EU_AI_Act_2024
Category: ARTICLE
Legal Unit: ARTICLE 10
Chunk Index: 1
Is Continuation: True
ner_LAW: ['ARTICLE 10']
ner_CARDINAL: ['2.', '3.']
ner_RISK_LEVEL: ['high-risk']
ner_GPE: ['Union']
ner_DOCUMENT: ['Regulation']

--- Text Preview ---
[ARTICLE 10] 2. Training, validation and testing data sets shall be subject to data governance and management practices appropriate for the intended purpose of the high-risk AI system. Those practices shall concern in particular: (a) the relevant design choices; (b) data collection processes and the origin of data, and in the case of personal data, the original purpose of the data collection; (c) relevant data-preparation processing operations, such as annotation, labelling, cleaning, updating, enrichment and aggregation; (d) the formulation of assumptions, in particular with respect to the information that the data are supposed to measure and represent; (e) an assessment of the availability, quantity and suitability of the data sets that

An additional refinement step was implemented to optimize the dataset for the Retrieval-Augmented Generation architecture. Following the initial text splitting, a filtering mechanism was applied to remove 7 low-value, boilerplate chunks, specifically legislative preambles and fragmented text containing fewer than 100 characters. Addiotionaly, leading punctuation artifacts generated by the recursive splitting process were stripped. This reduction from 577 to 570 highly concentrated text chunks ensures the FAISS vector database is populated exclusively with semantically dense legal provisions. By eliminating contextual noise, this approach maximizes embedding precision, reduces retrieval latency, and mitigates the risk of LLM hallucinations during the generation phase.

In [ ]:
polished_documents = []
deleted_chunks = []

for doc in final_documents:
    clean_text = re.sub(r"^(\[.*?\])\s*[,.]+\s*", r"\1 ", doc.page_content)

    if len(clean_text) > 100:
        doc.page_content = clean_text
        polished_documents.append(doc)
    else:
        doc.page_content = clean_text
        deleted_chunks.append(doc)

print(f"Original chunks: {len(final_documents)}")
print(f"Polished chunks ready for vectorization: {len(polished_documents)}")
print(f"Removed {len(deleted_chunks)} low-value chunks.\n")

Original chunks: 577
Polished chunks ready for vectorization: 570
Removed 7 low-value chunks.



In [ ]:
print(" DELETED CHUNKS PREVIEW ")
if not deleted_chunks:
    print("No chunks were deleted.")
else:
    for i, doc in enumerate(deleted_chunks):
        print(f"\n--- Deleted Chunk {i+1} ---")
        print(f"Legal Unit: {doc.metadata.get('legal_unit', 'Unknown')}")
        print(f"Length: {len(doc.page_content)} characters")
        print(f"Text: '{doc.page_content}'")

 DELETED CHUNKS PREVIEW 

--- Deleted Chunk 1 ---
Legal Unit: RECITAL 8
Length: 12 characters
Text: '[RECITAL 8] '

--- Deleted Chunk 2 ---
Legal Unit: RECITAL 35
Length: 31 characters
Text: '[RECITAL 35] Such data includes'

--- Deleted Chunk 3 ---
Legal Unit: ARTICLE 3
Length: 12 characters
Text: '[ARTICLE 3] '

--- Deleted Chunk 4 ---
Legal Unit: ARTICLE 5
Length: 37 characters
Text: '[ARTICLE 5] Prohibited AI practices 1'

--- Deleted Chunk 5 ---
Legal Unit: ARTICLE 13
Length: 13 characters
Text: '[ARTICLE 13] '

--- Deleted Chunk 6 ---
Legal Unit: ARTICLE 16
Length: 13 characters
Text: '[ARTICLE 16] '

--- Deleted Chunk 7 ---
Legal Unit: ARTICLE 66
Length: 13 characters
Text: '[ARTICLE 66] '


Running displacy one final time on the polished RAG chunks to prove that the custom entities defined in PATTERNS survived the text splitting, metadata injection, and string cleaning processes without being corrupted, and showcasing the accuracy reached by the ML model to identify entity from the AI act.

In [ ]:
sample_chunks = random.sample(polished_documents, 3)

for i, doc in enumerate(sample_chunks):
    print(f"\n--- CHUNK {i+1} (Source: {doc.metadata['legal_unit']}) ---")
    spacy_doc = nlp(doc.page_content)

    custom_colors = {
        "LEGAL_ROLE": "linear-gradient(90deg, #f1c40f, #f39c12)",
        "RISK_LEVEL": "linear-gradient(90deg, #e74c3c, #c0392b)",
        "CONCEPT": "linear-gradient(90deg, #3498db, #2980b9)",
        "DOCUMENT": "linear-gradient(90deg, #9b59b6, #8e44ad)",
        "LAW": "linear-gradient(90deg, #34495e, #2c3e50)",
        "GPE": "linear-gradient(90deg, #2ecc71, #27ae60)"
    }

    #  hides the 'IGNORE' tags
    visible_ents = ["ORG", "GPE", "DOCUMENT", "LAW", "CONCEPT", "LEGAL_ROLE", "RISK_LEVEL"]

    options = {
        "colors": custom_colors,
        "ents": visible_ents
    }

    displacy.render(spacy_doc, style="ent", jupyter=True, options=options)


--- CHUNK 1 (Source: ARTICLE 60) ---



--- CHUNK 2 (Source: ARTICLE 99) ---



--- CHUNK 3 (Source: ARTICLE 101) ---


**ENTITY TYPE DISTRIBUTION**


Not all extracted metadata provide the same level of value for retrieval and reasoning. Some fields are relatively generic or structurally redundant and may have limited impact on ranking performance, while others are particularly informative in a legal and regulatory domain. In particular, risk levels, explicit legal references (e.g. articles and paragraphs), and regulatory actors carry strong semantic and normative signals and can significantly improve filtering, disambiguation, and answer grounding in a RAG system. Nevertheless, all extracted metadata are retained, as they may become useful for future extensions.

In [ ]:
all_entity_counts = Counter()

for doc in polished_documents:
    for key, val_list in doc.metadata.items():
        if key.startswith("ner_"):
            clean_label = key.replace("ner_", "")
            all_entity_counts[clean_label] += len(val_list)

label_df = pd.DataFrame(all_entity_counts.most_common(), columns=['Entity Type', 'Count'])

fig = px.bar(
    label_df,
    x='Entity Type',
    y='Count',
    title='Named Entity Type Distribution (All Entities)',
    color='Count',
    color_continuous_scale='RdBu_r',
    text_auto=True
)

fig.update_layout(
    xaxis_title='Entity Type',
    yaxis_title='Frequency',
    xaxis_tickangle=-45,
    template='plotly_white',
    showlegend=False
)

fig.show()

Finally the chunks are ready and is possible to check the final lenght distribution.

In [ ]:
avg_len = sum(chunk_lengths) / len(chunk_lengths) if chunk_lengths else 0

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=chunk_lengths,
    nbinsx=50,
    marker_color='#e74c3c',
    marker_line=dict(color='white', width=1),
    name='Chunk Lengths'
))

fig.add_vline(x=1500, line_dash="dash", line_color="#c0392b")

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    line=dict(color='#c0392b', dash='dash'),
    name='Max Chunk Limit (1500)'
))

fig.add_vline(x=avg_len, line_dash="solid", line_color="#f39c12")


fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    line=dict(color='#f39c12'),
    name=f'Average Length ({avg_len:.0f})'
))

fig.update_layout(
    title='<b>Distribution of Final Vectorized Chunk Lengths</b>',
    xaxis_title="Characters per Chunk",
    yaxis_title="Number of Chunks",
    template='plotly_white',
    bargap=0.05,
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor="rgba(255, 255, 255, 0.5)"
    ),
    height=500
)


fig.show()

Below is shown a barplot of the absolute count of chunks belonging to RECITAL, ARTICLE, and ANNEX. This proves that the dataset is heavily weighted toward operative legal text (Articles) but retains the contextual intent (Recitals).

In [ ]:
categories = [doc.metadata['category'] for doc in polished_documents]
category_counts = Counter(categories)
sorted_categories = sorted(category_counts.items(), key=lambda x: x[1], reverse=True)

labels = [x[0] for x in sorted_categories]
values = [x[1] for x in sorted_categories]

fig = go.Figure(data=[
    go.Bar(
        x=labels,
        y=values,
        text=values,
        textposition='auto',
        marker=dict(
            color=values,
            colorscale=[
                [0, '#ffcccc'],
                [1, '#b30000']
            ],
            showscale=False,
            line=dict(color='white', width=1.5)
        )
    )
])

fig.update_layout(
    title='<b>Document Structural Balance</b><br><sup>Chunks per Category</sup>',
    xaxis_title="Category",
    yaxis_title="Number of Vectorized Chunks",
    template='plotly_white',
    font=dict(family="Arial", size=12),
    bargap=0.2,
    height=500
)


fig.show()

# TF-IDF EDA

The TF-IDF approach is an essential exploratory tool because it goes far beyond simple word counts, which can often be misleading. TF-IDF evaluates the importance of a word by balancing its local frequency within a specific chunk against its global rarity across the entire dataset.  It deliberately penalizes common words that appear everywhere and assigns greater weight to terms that uniquely characterize individual documents. This mathematical filtering ensures that words with higher TF-IDF scores genuinely represent the most significant and meaningful concepts of the text.

In [ ]:
corpus = [doc.page_content for doc in polished_documents]

custom_stop_words = ['shall', 'article', 'regulation', 'paragraph', 'annex',
                     'member', 'states', 'union', 'eu', 'referred', 'accordance', 'chapter', 'recital']

all_stop_words = list(ENGLISH_STOP_WORDS) + custom_stop_words

vectorizer = TfidfVectorizer(stop_words=all_stop_words, max_features=1000)
tfidf_matrix = vectorizer.fit_transform(corpus)

word_scores = tfidf_matrix.sum(axis=0).A1
words = vectorizer.get_feature_names_out()

df_words = pd.DataFrame({'word': words, 'score': word_scores})
top_20_words = df_words.sort_values(by='score', ascending=False).head(20)

top_20_words = top_20_words.sort_values(by='score', ascending=True)

fig = go.Figure()

fig.add_trace(go.Bar(
    y=top_20_words['word'],
    x=top_20_words['score'],
    orientation='h',
    marker=dict(
        color='#9b59b6',
        line=dict(color='white', width=1)
    ),
    name='TF-IDF Score'
))

fig.update_layout(
    title='<b>Top 20 Most Important Terms in the AI Act (TF-IDF)</b>',
    xaxis_title="Cumulative TF-IDF Score",
    yaxis_title="Terms",
    template='plotly_white',
    height=700,
    yaxis={'categoryorder':'total ascending'},
    margin=dict(l=150)
)

fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='rgba(0,0,0,0.1)')

fig.show()

# EMBEDDDING THE CHUNKS AND CREATION OF DATABASE

In [ ]:
!pip install sentence-transformers faiss-cpu

**BGE_M3**


It uses a shared vector space for over 100 languages. This is critical for the AI Act, as it allows a user to query in their native tongue and retrieve the precise corresponding legal section, even if the primary knowledge base is in English, BGE-M3 demonstrates state-of-the-art cross-lingual transfer.
By setting normalize_embeddings=True, the model's ability to weight important legal keywords in enhanced. In the context of the AI Act, this means the model "learns" that terms like "High-Risk" are more vital to the search result than common stop-words

In [ ]:
hf_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)
print("Model loaded")

print(f"Embedding {len(polished_documents)} chunks. This will take a moment...")

vectorstore = FAISS.from_documents(polished_documents, hf_embeddings) # text to gpu, vectorizes, and organizes in FAISS index
print("FAISS index built")

vectorstore.save_local(drive_persistence_path)

print("\n Vector database created and saved")

# RETRIVAL LOGIC

This search architecture uses LangChain’s EnsembleRetriever to combine the strengths of semantic and keyword-based retrieval methods. While standard FAISS vector search (dense retrieval) excels at understanding the contextual meaning of queries, it can sometimes miss exact matches, such as specific legal terms or article numbers. On the other hand, the BM25 algorithm (sparse retrieval) is highly precise for keyword matching but lacks semantic understanding.

By combining both approaches with equal influence (weights=[0.5, 0.5]), the system executes both searches in parallel and merges their outputs using Reciprocal Rank Fusion (RRF). This mathematical algorithm recalculates the score of each document based on its rank position in both individual lists. Consequently, legal chunks that contain both the exact terminology (captured by BM25) and the correct contextual meaning (captured by FAISS) are elevated to the top of the final results. This dual-engine approach drastically increases overall retrieval precision, ensuring the generative AI model is grounded in the most accurate and comprehensive legal context possible.

In [ ]:
!pip install rank_bm25

In [ ]:
# instanciating the model and loading database
hf_embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'},
    encode_kwargs={'normalize_embeddings': True}
)


vectorstore = FAISS.load_local(
    folder_path=drive_persistence_path,
    embeddings=hf_embeddings,
    allow_dangerous_deserialization=True
)
print("Database loaded")

In [ ]:
all_possible_ner_keys = set()
for doc in polished_documents:
    for key in doc.metadata.keys():
        if key.startswith("ner_"):
            all_possible_ner_keys.add(key)

all_possible_ner_keys = sorted(list(all_possible_ner_keys))

query = "What are the rules and obligations for high-risk biometric identification systems?"
print(f"User Question: '{query}'\n")

bm25_retriever = BM25Retriever.from_documents(polished_documents)
bm25_retriever.k = 3

faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

hybrid_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.5, 0.5] # 50% keyword, 50% semantic
)

top_chunks = hybrid_retriever.invoke(query)[:5]


for i, chunk in enumerate(top_chunks):
    print(f"RESULT {i+1} ")

    for key, value in chunk.metadata.items():
        if not key.startswith("ner_"):
            clean_key = key.replace('_', ' ').title()
            print(f" - {clean_key}: {value}")

    print(f"\nText Preview: {chunk.page_content[:500]}...\n")

    for ner_key in all_possible_ner_keys:
        display_name = ner_key.replace('ner_', '')
        extracted_entities = chunk.metadata.get(ner_key, [])
        print(f" - {display_name}: {extracted_entities}")
    print("\n")

User Question: 'What are the rules and obligations for high-risk biometric identification systems?'

RESULT 1 
 - Source: EU_AI_Act_2024
 - Category: RECITAL
 - Legal Unit: RECITAL 3
 - Chunk Index: 1
 - Is Continuation: True

Text Preview: [RECITAL 3] To the extent that this Regulation contains specific rules on the protection of individuals with regard to the processing of personal data concerning restrictions of the use of AI systems for remote biometric identification for the purpose of law enforcement, of the use of AI systems for risk assessments of natural persons for the purpose of law enforcement and of the use of AI systems of biometric categorisation for the purpose of law enforcement, it is appropriate to base this Regu...

 - CARDINAL: []
 - CONCEPT: []
 - DATE: []
 - DOCUMENT: ['Regulation']
 - EVENT: []
 - FAC: []
 - GPE: []
 - LAW: ['Article 16']
 - LEGAL_ROLE: []
 - LOC: []
 - NORP: []
 - ORDINAL: []
 - ORG: ['European Data Protection Board']
 - PERCENT: []
 - PERSON: 

**HYBRID RETRIVAL (METADATA-FILTERED)**

Standard search relies strictly on mathematical scoring (vector proximity and keyword frequency) to rank documents. While highly effective at finding general contextual relevance, it cannot guarantee that the retrieved documents adhere to specific factual constraints, such as ensuring a legal obligation strictly applies to a "deployer" rather than a "provider".
To solve this, the code below broadens the initial search space by retrieving the top 15 results from both the dense and sparse retrievers. Once the Reciprocal Rank Fusion algorithm merges and mathematically ranks these chunks into a list, the loop evaluates the pre-extracted Named Entity Recognition tags attached to each chunk's metadata. Any chunk that does not explicitly contain the target entity ("deployer") is immediately discarded, regardless of how high its mathematical search score was.
This architecture increases precision by combining the semantic intelligence of a standard search with the rigid, rules-based logic of metadata filtering. It eliminates the risk of "entity confusion" ensuring the generative LLM only reads legal rules that definitively apply to the exact stakeholder requested in the prompt.

In [ ]:
query = "What are the registration obligations for high-risk biometric systems?"
print(f"User question: {query}")

bm25_retriever.k = 15
faiss_retriever.search_kwargs = {"k": 15}

raw_hybrid_chunks = hybrid_retriever.invoke(query)

filtered_chunks = []
for chunk in raw_hybrid_chunks:
    legal_roles = chunk.metadata.get("ner_LEGAL_ROLE", [])

    if any("deployer" in role.lower() for role in legal_roles):
        filtered_chunks.append(chunk)

if not filtered_chunks:
    print("No chunks found with 'deployer' metadata.")


for i, chunk in enumerate(filtered_chunks):
    print(f"FILTERED HYBRID RESULT {i+1} ")

    for key, value in chunk.metadata.items():
        if not key.startswith("ner_"):
            clean_key = key.replace('_', ' ').title()
            print(f" - {clean_key}: {value}")

    print("\nTarget Metadata Found:")
    print(f" - Legal Roles: {chunk.metadata.get('ner_LEGAL_ROLE', [])}")

    print(f"\nText Preview: {chunk.page_content[:500]}...\n")

bm25_retriever.k = 3
faiss_retriever.search_kwargs = {"k": 3}

FILTERED HYBRID RESULT 1 
 - Source: EU_AI_Act_2024
 - Category: ARTICLE
 - Legal Unit: ARTICLE 26
 - Chunk Index: 3
 - Is Continuation: True

Target Metadata Found:
 - Legal Roles: ['Deployers', 'deployer', 'deployers', 'provider']

Text Preview: [ARTICLE 26] 8. Deployers of high-risk AI systems that are public authorities, or Union institutions, bodies, offices or agencies shall comply with the registration obligations referred to in Article 49. When such deployers find that the high-risk AI system that they envisage using has not been registered in the EU database referred to in Article 71, they shall not use that system and shall inform the provider or the distributor. 9. Where applicable, deployers of high-risk AI systems shall use t...

FILTERED HYBRID RESULT 2 
 - Source: EU_AI_Act_2024
 - Category: ARTICLE
 - Legal Unit: ARTICLE 26
 - Chunk Index: 4
 - Is Continuation: True

Target Metadata Found:
 - Legal Roles: ['deployer']

Text Preview: [ARTICLE 26] Each use shall be limite

# DATASET FOR CLASSIFICATION

The following is the prompt used to generate the synthetic dataset used to train the following classifiers, with the help of this LLM: Gemini 3 (reasoning)

The prompt:
You are an expert AI data engineer and EU legal scholar. I am fine-tuning a DistilBERT text classifier and need a highly diverse synthetic dataset of AI product descriptions mapped to their official EU AI Act risk levels.
Generate exactly 100 rows of data in perfect CSV format.
The Rules:

The CSV must have exactly two columns: text and label.
The text column should be a 1 to 2 sentence description of a fictional AI system or startup product. Enclose every text description in double quotes ("").
The label column must be EXACTLY one of these four strings (no spaces, case-sensitive): Unacceptable_Risk, High_Risk, Limited_Risk, Minimal_Risk.
Generate exactly 25 examples for each of the 4 categories.
Crucial: Maximize industry diversity! Do not just use medical or HR examples. Invent systems for agriculture, toys, deepfakes, law enforcement, education, banking, video games, aviation, and chatbots.
Do not include any markdown formatting like ```csv or conversational text in your response. Output ONLY the raw CSV text starting with the header row, so I can copy and paste it directly.

# DISTILBERT CLASSIFIER

In [ ]:
!pip install transformers datasets evaluate accelerate -U

Synthetic dataset loading:

In [45]:
file_path = "/content/drive/MyDrive/compliance_and_regulation_rag/ai_risk_dataset.csv.txt"

df = pd.read_csv(file_path)
print(df.head())
print(df['label'].value_counts())

                                                text              label
0  An AI-powered social scoring system used by mu...  Unacceptable_Risk
1  A real-time biometric identification system de...  Unacceptable_Risk
2  An AI application designed to subconsciously m...  Unacceptable_Risk
3  A predictive policing tool used to identify an...  Unacceptable_Risk
4  A cognitive-behavioral manipulation AI used by...  Unacceptable_Risk
label
Unacceptable_Risk    75
High_Risk            75
Limited_Risk         75
Minimal_Risk         75
Name: count, dtype: int64


Defining train, val, test split, performance metrics and training (fine-tuning) parameters:

In [ ]:
label_map = {"Minimal_Risk": 0, "Limited_Risk": 1, "High_Risk": 2, "Unacceptable_Risk": 3}
df['label'] = df['label'].map(label_map)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)
test_dataset = Dataset.from_pandas(test_df)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_train = train_dataset.map(tokenize_function, batched=True) # batched=True process faster
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    probabilities = softmax(logits, axis=-1)

    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')

    auc = roc_auc_score(labels, probabilities, multi_class='ovr', average='macro')

    # nir
    largest_class_count = max(np.bincount(labels))
    nir = largest_class_count / len(labels)

    return {
        "accuracy": accuracy,
        "f1_macro": f1,
        "recall_macro": recall,
        "roc_auc": auc,
        "NIR": nir
    }

training_args = TrainingArguments(
    output_dir="./ai_risk_classifier",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
)

In [ ]:
print(f"Train: {len(train_df)} | Val: {len(eval_df)} | Test: {len(test_df)}")

Train: 240 | Val: 30 | Test: 30


Training and saving the best model (based on validation loss)

In [ ]:
trainer.train()

print("\n test set evaluation")
test_results = trainer.predict(tokenized_test)

save_path = "/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/distilBERT_classifier"

trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)
print("\nTraining completed, Architecture saved.")

In [ ]:
log_history = trainer.state.log_history

metrics_per_epoch = {}

for entry in log_history:
    if "epoch" not in entry:
        continue

    epoch = round(entry["epoch"], 2)

    if epoch not in metrics_per_epoch:
        metrics_per_epoch[epoch] = {'Epoch': int(epoch)}

    if "loss" in entry and "eval_loss" not in entry:
        metrics_per_epoch[epoch]["Training Loss"] = entry["loss"]

    if "eval_loss" in entry:
        metrics_per_epoch[epoch]["Validation Loss"] = entry["eval_loss"]
        metrics_per_epoch[epoch]["Accuracy"] = entry.get("eval_accuracy")
        metrics_per_epoch[epoch]["F1 Macro"] = entry.get("eval_f1_macro")
        metrics_per_epoch[epoch]["Recall Macro"] = entry.get("eval_recall_macro")
        metrics_per_epoch[epoch]["Roc Auc"] = entry.get("eval_roc_auc")
        metrics_per_epoch[epoch]["Nir"] = entry.get("eval_NIR")

metrics_df = pd.DataFrame(list(metrics_per_epoch.values()))

metrics_df = metrics_df.dropna(subset=["Validation Loss"])

columns_order = [
    "Epoch", "Training Loss", "Validation Loss",
    "Accuracy", "F1 Macro", "Recall Macro", "Roc Auc", "Nir"
]
existing_columns = [col for col in columns_order if col in metrics_df.columns]
metrics_df = metrics_df[existing_columns]

print(metrics_df.to_string(index=False))

In [ ]:
log_history = trainer.state.log_history
data = []

for entry in log_history:
    if "loss" in entry and "eval_loss" not in entry:
        data.append({
            'Epoch': entry["epoch"],
            'Loss': entry["loss"],
            'Dataset': 'Training Loss'
        })


    if "eval_loss" in entry:
        data.append({
            'Epoch': entry["epoch"],
            'Loss': entry["eval_loss"],
            'Dataset': 'Validation Loss'
        })

df_base = pd.DataFrame(data)

if df_base[df_base['Dataset'] == 'Training Loss'].empty:
    print(" WARNING: No Training Loss data found in logs.")


all_frames = sorted(df_base['Epoch'].unique())
animated_data = []

for current_frame in all_frames:
    df_snapshot = df_base[df_base['Epoch'] <= current_frame].copy()
    df_snapshot['Frame'] = current_frame
    animated_data.append(df_snapshot)

df_animated = pd.concat(animated_data, ignore_index=True)


fig = px.line(
    df_animated,
    x="Epoch",
    y="Loss",
    color="Dataset",
    animation_frame="Frame",
    animation_group="Dataset",
    markers=True,
    title="Evolution of Training vs. Validation Loss",
    template="plotly_white",
    color_discrete_map={"Training Loss": "blue", "Validation Loss": "orange"},
    range_x=[0, df_base['Epoch'].max() * 1.05],
    range_y=[0, df_base['Loss'].max() * 1.1]
)

fig.show()

Performance report of DISTILBERT:

In [ ]:
print(f"Accuracy: {test_results.metrics['test_accuracy'] * 100:.2f}%")
print(f"F1 Score: {test_results.metrics['test_f1_macro']:.4f}")
print(f"Recall:   {test_results.metrics['test_recall_macro']:.4f}")
print(f"ROC AUC:  {test_results.metrics['test_roc_auc']:.4f}")
print(f"NIR:      {test_results.metrics['test_NIR'] * 100:.2f}%")

Accuracy: 90.00%
F1 Score: 0.8995
Recall:   0.8973
ROC AUC:  0.9866
NIR:      26.67%


In [ ]:
# loading the model back
load_path = "/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/distilBERT_classifier"

distilbert_model = AutoModelForSequenceClassification.from_pretrained(load_path)
tokenizer = AutoTokenizer.from_pretrained(load_path)

tokenized_test = tokenized_test_dataset.map(
    lambda x: tokenizer(x['text'], padding="max_length", truncation=True),
    batched=True
)

trainer = Trainer(model=distilbert_model)
test_results = trainer.predict(tokenized_test)

# RNN (GRU BASED)

Loading the data and using the same train, val, test split as the previus distilbert model to ensure consistency for the following performance comparison.

In [23]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

--2026-02-22 10:10:45--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2026-02-22 10:10:45--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2026-02-22 10:10:45--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [21]:
file_path = "/content/drive/MyDrive/compliance_and_regulation_rag/ai_risk_dataset.csv.txt"

df = pd.read_csv(file_path)

label_map = {"Minimal_Risk": 0, "Limited_Risk": 1, "High_Risk": 2, "Unacceptable_Risk": 3}
df['label'] = df['label'].map(label_map)

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
eval_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])


X_train, y_train = train_df['text'].to_numpy(), train_df['label'].to_numpy()
X_eval, y_eval = eval_df['text'].to_numpy(), eval_df['label'].to_numpy()
X_test, y_test = test_df['text'].to_numpy(), test_df['label'].to_numpy()

In [24]:
embeddings_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32') # embddings
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} pre-trained word vectors.")

Found 400000 pre-trained word vectors.


In [33]:
from tensorflow.keras.layers import TextVectorization
max_features = 1500
sequence_length = 100

vectorizer = TextVectorization(
    max_tokens=max_features,
    output_sequence_length=sequence_length
)

vectorizer.adapt(X_train)

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))
num_tokens = len(voc)
embedding_dim = 100 #  match the GloVe file downloaded (100d)

embedding_matrix = np.zeros((num_tokens, embedding_dim))

hits = 0
misses = 0
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1

print(f"Converted {hits} words ({misses} misses)")

Converted 1452 words (48 misses)


In [34]:
missing_words = []

for word in word_index.keys():
    if word not in embeddings_index:
        missing_words.append(word)

print("missing words:", missing_words)

missing words: ['', '[UNK]', np.str_('aipowered'), np.str_('aidriven'), np.str_('chatbot'), np.str_('deepfake'), np.str_('noisecanceling'), np.str_('spellcheck'), np.str_('nonplayer'), np.str_('highstakes'), np.str_('highrisk'), np.str_('highinterest'), np.str_('userselected'), np.str_('timeandattendance'), np.str_('texttospeech'), np.str_('settop'), np.str_('riskbased'), np.str_('recognitionlinked'), np.str_('nonrepeating'), np.str_('neurodivergent'), np.str_('microsleep'), np.str_('microexpressions'), np.str_('massscale'), np.str_('lowvalue'), np.str_('lofi'), np.str_('librarys'), np.str_('industrialscale'), np.str_('highvalue'), np.str_('hightraffic'), np.str_('highresolution'), np.str_('governmentlinked'), np.str_('governmentled'), np.str_('governmentbacked'), np.str_('gamified'), np.str_('firsttime'), np.str_('fearbased'), np.str_('falsefriend'), np.str_('earlystage'), np.str_('deprioritizes'), np.str_('deeplearning'), np.str_('countryoforigin'), np.str_('constellationmapping'), n

In [36]:
from tensorflow.keras.layers import SpatialDropout1D
model = Sequential([
    Input(shape=(1,), dtype=tf.string),
    vectorizer,

    Embedding(
        input_dim=num_tokens,
        output_dim=embedding_dim,
        embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
        trainable=False,
    ),

    SpatialDropout1D(0.2),

    Bidirectional(
        GRU(
            32,
            dropout=0.2,
            recurrent_dropout=0.1,
            return_sequences=False
        )
    ),

    Dense(32, activation='relu'),
    Dropout(0.3),

    Dense(4, activation='softmax')
])


total_steps = (len(X_train) // 8) * 10 # epochs = 10

# linear decay
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.0,
    decay_steps=total_steps,
    power=1.0
)


model.compile(
    optimizer=AdamW(learning_rate=lr_scheduler, weight_decay=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_eval, y_eval),
    epochs=10,
    batch_size=8,
    callbacks=callbacks
)

In [38]:
metrics_dict = history.history
metrics_df = pd.DataFrame(metrics_dict)
metrics_df.insert(0, 'Epoch', range(1, len(metrics_df) + 1))
print(metrics_df.to_string(index=False))

 Epoch  accuracy     loss  val_accuracy  val_loss
     1  0.220833 1.431748      0.233333  1.390280
     2  0.250000 1.412477      0.233333  1.386917
     3  0.275000 1.397859      0.233333  1.384846
     4  0.220833 1.430615      0.233333  1.383367
     5  0.300000 1.405123      0.233333  1.381959
     6  0.225000 1.399086      0.233333  1.381130
     7  0.266667 1.395372      0.233333  1.380363
     8  0.312500 1.377813      0.233333  1.380013
     9  0.250000 1.388897      0.233333  1.379462
    10  0.262500 1.401886      0.233333  1.379342


In [39]:
# The model.fit() process in Keras returns a history object.
# extract the training and validation losses from it:
train_loss = history.history['loss']
eval_loss = history.history['val_loss']

# Keras epochs are 0-indexed
epochs = [e + 1 for e in history.epoch]

data = []

for e, l in zip(epochs, train_loss):
    data.append({'Epoch': e, 'Loss': l, 'Dataset': 'Training Loss'})
for e, l in zip(epochs, eval_loss):
    data.append({'Epoch': e, 'Loss': l, 'Dataset': 'Validation Loss'})

df_base = pd.DataFrame(data)

animated_data = []
unique_epochs = sorted(list(set(epochs)))


for current_frame in unique_epochs:
    df_snapshot = df_base[df_base['Epoch'] <= current_frame].copy()
    df_snapshot['Frame'] = current_frame
    animated_data.append(df_snapshot)

df_animated = pd.concat(animated_data, ignore_index=True)

fig = px.line(
    df_animated,
    x="Epoch",
    y="Loss",
    color="Dataset",
    animation_frame="Frame",
    animation_group="Dataset",
    markers=True,
    range_x=[0, df_base['Epoch'].max() + 0.2],
    range_y=[df_base['Loss'].min() * 0.9, df_base['Loss'].max() * 1.1],
    title="Evolution of Training vs. Validation Loss (GRU Model)",
    template="plotly_white",
    color_discrete_map={"Training Loss": "blue", "Validation Loss": "orange"}
)

fig.show()

In [ ]:
probabilities = model.predict(X_test)
predictions = np.argmax(probabilities, axis=-1)

accuracy = accuracy_score(y_test, predictions)
f1 = f1_score(y_test, predictions, average='macro')
recall = recall_score(y_test, predictions, average='macro')
auc = roc_auc_score(y_test, probabilities, multi_class='ovr', average='macro')

largest_class_count = max(np.bincount(y_test))
nir = largest_class_count / len(y_test)

results = {
    "accuracy": accuracy,
    "f1_macro": f1,
    "recall_macro": recall,
    "roc_auc": auc,
    "NIR": nir
}

In [41]:
for metric, value in results.items():
    print(f"{metric}: {value:.4f}")

save_path = "/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/gru_model.keras"
model.save(save_path)

print("\nTraining complete, Architecture saved.")

accuracy: 0.2667
f1_macro: 0.1053
recall_macro: 0.2500
roc_auc: 0.6230
NIR: 0.2667

Training complete, Architecture saved.


In [43]:
# loading the model
from tensorflow.keras.models import load_model
gru_model_path = "/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/gru_model.keras"
gru_model = load_model(gru_model_path)

# RANDOM FOREST CLASSIFIER

In [18]:
X_train, X_temp, y_train, y_temp = train_test_split(
    df['text'],
    df['label'],
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

X_eval, X_test, y_eval, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=1500, stop_words='english')),
    ('classifier', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

param_grid = {
    'classifier__n_estimators': [100, 200, 300],

    'classifier__max_depth': [10, 20, 30, None],

    'classifier__max_features': ['sqrt', 'log2', 0.15, 0.2]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("\n=== Grid Search Complete ===")
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation F1-Macro: {grid_search.best_score_:.4f}\n")

best_rf_model = grid_search.best_estimator_
predictions = best_rf_model.predict(X_test)

print("=== Best Model Performance on Test Set ===")
print(classification_report(y_test, predictions))


=== Grid Search Complete ===
Best Parameters: {'classifier__max_depth': 20, 'classifier__max_features': 'log2', 'classifier__n_estimators': 100}
Best Cross-Validation F1-Macro: 0.7475

=== Best Model Performance on Test Set ===
                   precision    recall  f1-score   support

        High_Risk       1.00      0.38      0.55         8
     Limited_Risk       0.83      0.71      0.77         7
     Minimal_Risk       0.44      0.88      0.58         8
Unacceptable_Risk       0.80      0.57      0.67         7

         accuracy                           0.63        30
        macro avg       0.77      0.63      0.64        30
     weighted avg       0.76      0.63      0.64        30



In [19]:
# save
import joblib

rf_save_path = "/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/best_rf_model.pkl"
joblib.dump(best_rf_model, rf_save_path)

['/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/best_rf_model.pkl']

In [20]:
# load model

rf_load_path = "/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/best_rf_model.pkl"
best_rf_model = joblib.load(rf_load_path)

# SVM CLASSIFIER

In [15]:
X_train, X_temp, y_train, y_temp = train_test_split(
    df['text'],
    df['label'],
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

X_eval, X_test, y_eval, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier', SVC(kernel='linear', class_weight='balanced', random_state=42, probability = True))
])

param_grid = {
    # different levels of strictness for the SVM
    'classifier__C': [0.1, 1, 10, 100],

    # different vocabulary sizes
    'tfidf__max_features': [1000, 1500, 2000],

    # Unigrams (1,1) vs Unigrams + Bigrams (1,2)
    'tfidf__ngram_range': [(1, 1), (1, 2)]
}

grid_search_svm = GridSearchCV(
    estimator=svm_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)

grid_search_svm.fit(X_train, y_train)

print("\n=== SVM Grid Search Complete ===")
print(f"Best Parameters: {grid_search_svm.best_params_}")
print(f"Best Cross-Validation F1-Macro: {grid_search_svm.best_score_:.4f}\n")

best_svm_model = grid_search_svm.best_estimator_
predictions = best_svm_model.predict(X_test)

print("=== Best SVM Model Performance on Test Set ===")
print(classification_report(y_test, predictions))

Fitting 5 folds for each of 24 candidates, totalling 120 fits

=== SVM Grid Search Complete ===
Best Parameters: {'classifier__C': 10, 'tfidf__max_features': 1500, 'tfidf__ngram_range': (1, 1)}
Best Cross-Validation F1-Macro: 0.8787

=== Best SVM Model Performance on Test Set ===
                   precision    recall  f1-score   support

        High_Risk       0.86      0.75      0.80         8
     Limited_Risk       0.83      0.71      0.77         7
     Minimal_Risk       0.70      0.88      0.78         8
Unacceptable_Risk       0.71      0.71      0.71         7

         accuracy                           0.77        30
        macro avg       0.78      0.76      0.77        30
     weighted avg       0.78      0.77      0.77        30



In [16]:
# save
svm_save_path = "/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/best_svm_model.pkl"
joblib.dump(best_svm_model, svm_save_path)

['/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/best_svm_model.pkl']

In [17]:
#load
svm_load_path = "/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/best_svm_model.pkl"
best_svm_model = joblib.load(svm_load_path)

# XGBOOST CLASSIFIER

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

X_train, X_temp, y_train, y_temp = train_test_split(
    df['text'],
    df['label'],
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

X_eval, X_test, y_eval, y_test = train_test_split(
    X_temp,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)


xgb_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english')),
    ('classifier', XGBClassifier(random_state=42))
])

param_grid = {
    # XGBoost  parameters
    'classifier__learning_rate': [0.05, 0.1, 0.2],
    'classifier__max_depth': [1,2,3],         # depth trees
    'classifier__n_estimators': [100, 200, 300],     # number sequential trees

    # TF-IDF parameters
    'tfidf__max_features': [1000, 1500, 2000],
    'tfidf__ngram_range': [(1, 1), (1, 2)]
}


grid_search_xgb = GridSearchCV(
    estimator=xgb_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,
    verbose=1
)


grid_search_xgb.fit(X_train, y_train_encoded)


print("\n=== XGBoost Grid Search Complete ===")
print(f"Best Parameters: {grid_search_xgb.best_params_}")
print(f"Best Cross-Validation F1-Macro: {grid_search_xgb.best_score_:.4f}\n")


best_xgb_model = grid_search_xgb.best_estimator_
predictions_encoded = best_xgb_model.predict(X_test)

predictions = le.inverse_transform(predictions_encoded)

Fitting 5 folds for each of 162 candidates, totalling 810 fits

=== XGBoost Grid Search Complete ===
Best Parameters: {'classifier__learning_rate': 0.05, 'classifier__max_depth': 1, 'classifier__n_estimators': 200, 'tfidf__max_features': 1000, 'tfidf__ngram_range': (1, 2)}
Best Cross-Validation F1-Macro: 0.6316



In [10]:
print("=== Best XGBoost Model Performance on Test Set ===")
print(classification_report(y_test, predictions))

=== Best XGBoost Model Performance on Test Set ===
                   precision    recall  f1-score   support

        High_Risk       0.62      0.62      0.62         8
     Limited_Risk       1.00      0.43      0.60         7
     Minimal_Risk       0.50      0.88      0.64         8
Unacceptable_Risk       0.60      0.43      0.50         7

         accuracy                           0.60        30
        macro avg       0.68      0.59      0.59        30
     weighted avg       0.67      0.60      0.59        30



In [13]:
import joblib
xgb_save_path = "/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/best_xgb_model.pkl"
joblib.dump(best_xgb_model, xgb_save_path)

['/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/best_xgb_model.pkl']

In [14]:
#load

xgb_load_path = "/content/drive/MyDrive/compliance_and_regulation_rag/classification_results/best_xgb_model.pkl"
best_xgb_model = joblib.load(xgb_load_path)

# FINAL COMPARISON OF CLASSIFIERS

In [ ]:
y_pred_rf = best_rf_model.predict(X_test)
y_prob_rf = best_rf_model.predict_proba(X_test)

y_pred_svm = best_svm_model.predict(X_test)
y_prob_svm = best_svm_model.predict_proba(X_test)

y_pred_xgb = best_xgb_model.predict(X_test)
y_prob_xgb = best_xgb_model.predict_proba(X_test)


def get_ml_metrics(y_true, y_pred, y_prob):
    return {
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1 Macro": f1_score(y_true, y_pred, average='macro'),
        "Recall Macro": recall_score(y_true, y_pred, average='macro'),
        "ROC AUC": roc_auc_score(y_true, y_prob, multi_class='ovr', average='macro'),
        "NIR": 0.2667
    }

comparison_data = [
    {
        "Model": "DistilBERT (Pre-trained)",
        "Accuracy": test_results.metrics['test_accuracy'],
        "F1 Macro": test_results.metrics['test_f1_macro'],
        "Recall Macro": test_results.metrics['test_recall_macro'],
        "ROC AUC": test_results.metrics['test_roc_auc'],
        "NIR": test_results.metrics['test_NIR']
    },
    {
        "Model": "Micro-GRU (Baseline)",
        "Accuracy": results['accuracy'],
        "F1 Macro": results['f1_macro'],
        "Recall Macro": results['recall_macro'],
        "ROC AUC": results['roc_auc'],
        "NIR": results['NIR']
    }
]

rf_metrics = get_ml_metrics(y_test, y_pred_rf, y_prob_rf)
rf_metrics["Model"] = "Random Forest (Tuned)"
comparison_data.append(rf_metrics)

svm_metrics = get_ml_metrics(y_test, y_pred_svm, y_prob_svm)
svm_metrics["Model"] = "Linear SVM (Tuned)"
comparison_data.append(svm_metrics)

xgb_metrics = get_ml_metrics(y_test, y_pred_xgb, y_prob_xgb)
xgb_metrics["Model"] = "XGBoost (Tuned)"
comparison_data.append(xgb_metrics)

df_compare = pd.DataFrame(comparison_data)

cols = ["Model", "Accuracy", "F1 Macro", "Recall Macro", "ROC AUC", "NIR"]
df_compare = df_compare[cols]

format_dict = {
    "Accuracy": "{:.2%}",
    "F1 Macro": "{:.4f}",
    "Recall Macro": "{:.4f}",
    "ROC AUC": "{:.4f}",
    "NIR": "{:.2%}"
}

display(df_compare.style.format(format_dict))

# GIT PUSH

In [ ]:
# for pushing
!git add .
!git commit -m "env created"
!git push -u origin main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
Branch 'main' set up to track remote branch 'main' from 'origin'.
Everything up-to-date
